# ACPF data preparation process
Md Bodruddoza (Zion), 
PhD Candidate, University of Guelph, Ontario
Email: mbodrudd@uoguelph.ca

The steps below are coded in ArcPro to automate the data preparation process for running the ACPF toolbox for Ontario.
It is important to organize the base data before running the codes.

Part-1: Watershed boundary

1.	Open an ArcPro project, connect to a folder, create a geodatabase using the naming convention of ACPF, set the Environment 
2.	Load ON_Watersheds file and select watershed boundary using an OGFID 
3.	Export the watershed boundary in a Geodatabase and give a name as ontOGFID
4.	Use the watershed boundary to select your field boundary using select by location tool
5.	Export the selected field layer to the ACPF geodatabase
6.	Edit the field boundary by identifying and deleting small parcels from the layer
7.	Make a union file including the edited field layer and the watershed boundary
8.	Make a buffer watershed boundary from the union layer. Use dissolve all output features into a single feature 

Part-2: Landuse

1.	Extracting CDL layers and attribute table preparation 
2.	Update edited field boundary 

Part-3: Soil

1.	Extract the soil raster according to your buffer watershed boundary
2.	Join the raster attribute table with ONSCC_MUKEYdata to include relevant parameters in your raster file
3.	Export the raster attribute table and name it as SoilCMPLX
4.	Join the exported and modified SoilCmplex table with ONSCC_CMPLXdata using MUKEY. Clean the table by removing unwanted fields


# Extracting watershed boundary, field boundary and creating buffer watershed 

In [1]:
import arcpy
import os

# Define the base directory #***Need to change according to the data path and ogf_id
base_dir = r"D:\ACPFDataRequest\OGFIDrelatedData"
ogf_id = "135167370"  # OGF_ID from the watershed boundary shapefile

# Paths to source data
base_data_gdb = os.path.join(base_dir, "ACPFOntario_BaseData.gdb")
soils_data_gdb = os.path.join(base_dir, "ACPFOntario_Soils.gdb")
raster_dir = os.path.join(base_dir, "wsCDL_raster_files")

# Create target geodatabase if it doesn't exist
target_gdb_name = f"ont{ogf_id}.gdb"
target_gdb_path = os.path.join(base_dir, "OGFIDbasedData", target_gdb_name)
if not arcpy.Exists(target_gdb_path):
    arcpy.CreateFileGDB_management(os.path.dirname(target_gdb_path), os.path.basename(target_gdb_path))

# Set ArcGIS environment settings
arcpy.env.workspace = target_gdb_path
arcpy.env.scratchWorkspace = target_gdb_path
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 UTM Zone 17N")

# Function to simplify feature export
def export_feature(source_feature, target_feature_name, where_clause=""):
    arcpy.conversion.ExportFeatures(
        in_features=source_feature,
        out_features=os.path.join(target_gdb_path, target_feature_name),
        where_clause=where_clause,
        use_field_alias_as_name="NOT_USE_ALIAS"
    )

# Select and export watershed boundary
watershed_feature = os.path.join(base_data_gdb, "ON_Watersheds")
export_feature(watershed_feature, f"bndont{ogf_id}", where_clause=f"OGF_ID = {ogf_id}")

# Select and export field boundaries using the exported watershed boundary for selection
# First, make a feature layer from the field boundaries
fields_feature = os.path.join(base_data_gdb, "ON_Fields")
arcpy.MakeFeatureLayer_management(fields_feature, "fields_layer")

# Select fields that intersect with the exported watershed boundary
arcpy.management.SelectLayerByLocation(
    in_layer="fields_layer",
    overlap_type="INTERSECT",
    select_features=os.path.join(target_gdb_path, f"bndont{ogf_id}"),
    selection_type="NEW_SELECTION"
)

# Export the selected fields to the target geodatabase
export_feature("fields_layer", f"FBont{ogf_id}draft")

# Delete small parcels
field_boundaries_draft = os.path.join(target_gdb_path, f"FBont{ogf_id}draft")
arcpy.MakeFeatureLayer_management(field_boundaries_draft, "draft_fields_layer")  # Make layer for selection
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="draft_fields_layer",
    selection_type="NEW_SELECTION",
    where_clause="Shape_Area <= 20000"
)
arcpy.management.DeleteFeatures("draft_fields_layer")

# Perform union
union_output = f"bndont{ogf_id}_Union"
arcpy.analysis.Union(
    in_features=[os.path.join(target_gdb_path, f"bndont{ogf_id}"), "draft_fields_layer"],
    out_feature_class=os.path.join(target_gdb_path, union_output),
    join_attributes="ALL",
    gaps="GAPS"
)

# Create buffer
buffer_output = f"bufont{ogf_id}"
arcpy.analysis.Buffer(
    in_features=os.path.join(target_gdb_path, union_output),
    out_feature_class=os.path.join(target_gdb_path, buffer_output),
    buffer_distance_or_field="1000 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL"
)

print("Watershed boundary, buffer watershed boundary, and draft field boundary have been created for further processing.")


Watershed boundary, buffer watershed boundary, and draft field boundary have been created for further processing.


# Crop data layer preparation and updating field boundary

In [2]:
import arcpy
import os

# Enable the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Define paths #***Need to change according to the data path and mask_data name
raster_data_folder = r"D:\ACPFDataRequest\OGFIDrelatedData\wsCDL_raster_files"
output_gdb_path = r"D:\ACPFDataRequest\OGFIDrelatedData\OGFIDbasedData\ont135167370.gdb"
lookup_table_path = r"D:\ACPFDataRequest\OGFIDrelatedData\ACPFOntario_BaseData.gdb\ON_LookUpTable"
mask_data = "bufont135167370"

# Set environment settings
arcpy.env.workspace = raster_data_folder
arcpy.env.scratchWorkspace = output_gdb_path
arcpy.env.overwriteOutput = True

# Extract and join
raster_files = arcpy.ListRasters("*", "TIF")
for raster in raster_files:
    year = raster.split('_')[1]
    output_raster_name = f"wsCDL{year}"
    output_raster_path = os.path.join(output_gdb_path, output_raster_name)
    
    # Extract raster
    out_raster = arcpy.sa.ExtractByMask(in_raster=raster, in_mask_data=mask_data)
    out_raster.save(output_raster_path)
    
    # Ensure the raster has an attribute table for joining
    arcpy.BuildRasterAttributeTable_management(out_raster, "Overwrite")
    
    # Perform join
    arcpy.management.JoinField(in_data=output_raster_path, in_field="Value", join_table=lookup_table_path,
                               join_field="Value", fields="CLASS_NAME;ROTVAL;PrimeName;OneName;isAG")
    
    # Optionally, remove irrelevant fields from the raster's attribute table
    # List fields and decide which to remove after join, example:
    # arcpy.DeleteField_management(output_raster_path, ["UnwantedField1", "UnwantedField2"])

    print(f"Processed and joined {raster}")

# Check in the Spatial Analyst extension
arcpy.CheckInExtension("Spatial")

print("All rasters processed and joined.")


Processed and joined aci_2017_on.tif
Processed and joined aci_2018_on.tif
Processed and joined aci_2019_on.tif
Processed and joined aci_2020_on.tif
Processed and joined aci_2021_on.tif
Processed and joined aci_2022_on.tif
All rasters processed and joined.


In [3]:
import arcpy

# Path to the ACPF toolbox #***Need to change according to toolbox path
toolbox_path = r"D:\ACPFDataRequest\OGFIDrelatedData\ACPF_V5_Pro_04042023\acpf_V5_Pro.atbx"
# Import the custom toolbox
arcpy.ImportToolbox(toolbox_path)

# Variables for clarity #***Need to change according to the data path
fb_feature_class = r"D:\ACPFDataRequest\OGFIDrelatedData\OGFIDbasedData\ont135167370.gdb\FBont135167370draft"
lookup_table = r"D:\ACPFDataRequest\OGFIDrelatedData\ACPFOntario_BaseData.gdb\ON_LookUpTable"
output_geodatabase = r"D:\ACPFDataRequest\OGFIDrelatedData\OGFIDbasedData\ont135167370.gdb"  # Explicit for clarity

# Execute the updateFieldBoundaries tool with specified parameters
arcpy.acpfV5ProATBX.updateFieldBoundaries(
    fb_feature_class,  # Path to the Edited Field Boundary feature class
    "10",  # Minimum feature size in acres
    lookup_table,  # Path to the ACPF Landuse Lookup table
    "2024"  # Field Boundary Update Year
)

# Note: The output will be directed to the specified geodatabase in the fb_feature_class path

print("Updated the field boundary, landuse6 table and crop history table created. Please check the target geodatabase")



Updated the field boundary, landuse6 table and crop history table created. Please check the target geodatabase


# Soil data preparation 

In [4]:
import arcpy

# Configuration for easy path and parameter adjustments
config = {
    "soil_gdb": r"D:\ACPFDataRequest\OGFIDrelatedData\ACPFOntario_Soils.gdb",
    "output_gdb": r"D:\ACPFDataRequest\OGFIDrelatedData\OGFIDbasedData\ont135167370.gdb",
    "buffer_feature_name": "bufont135167370",
    "soil_raster_name": "ONSSC",
    "mukey_data_name": "ONSSC_MUKEYdata",
    "complex_data_name": "ONSSC_CMPLXdata",
    "extracted_soil_raster_name": "gONSSCsoils",
    "soil_complex_table_name": "SoilCMPLXont135167370"
}

def extract_soil_raster():
    """Extracts the soil raster by a buffer feature and saves the output."""
    arcpy.env.scratchWorkspace = config["output_gdb"]
    in_raster = f"{config['soil_gdb']}\\{config['soil_raster_name']}"
    in_mask_data = f"{config['output_gdb']}\\{config['buffer_feature_name']}"
    out_raster = arcpy.sa.ExtractByMask(in_raster, in_mask_data)
    out_raster.save(f"{config['output_gdb']}\\{config['extracted_soil_raster_name']}")

def join_raster_attribute():
    """Joins the soil raster attribute table with lookup data to include relevant parameters."""
    in_data = f"{config['output_gdb']}\\{config['extracted_soil_raster_name']}"
    join_table = f"{config['soil_gdb']}\\{config['mukey_data_name']}"
    arcpy.management.JoinField(in_data, "MUKEY", join_table, "MUKEY", 
                               "SOIL_ID;HydroGrp;DrainCls;Hydric;kwfact_r;Coarse50_150;OM0_100;totalSand;WTDepAprJun")

def export_raster_table():
    """Exports the raster attribute table to the target geodatabase."""
    in_table = f"{config['output_gdb']}\\{config['extracted_soil_raster_name']}"
    out_table = f"{config['output_gdb']}\\{config['soil_complex_table_name']}"
    arcpy.conversion.ExportTable(in_table, out_table, use_field_alias_as_name="NOT_USE_ALIAS")

def join_soil_complex_data():
    """Joins the exported SoilComplex table with more detailed data using MUKEY."""
    in_data = f"{config['output_gdb']}\\{config['soil_complex_table_name']}"
    join_table = f"{config['soil_gdb']}\\{config['complex_data_name']}"
    arcpy.management.JoinField(in_data, "MUKEY", join_table, "MUKEY", fields=None)  # Simplified for brevity

def remove_unwanted_fields():
    """Removes specified fields from the soil complex table."""
    fields_to_remove = [
        "Value", "Count", "OGF_ID", "MUKEY_1", "HECTARES", "ACRES", 
        "LOCATION_A", "GEOMETRY_U", "EFFECTIVE_", "SYSTEM_DAT", 
        "OBJECTID_1", "SHAPEAREA", "SHAPELEN", "Shape_Length", "Shape_Area"
    ]
    soil_complex_table_path = f"{config['output_gdb']}\\{config['soil_complex_table_name']}"
    arcpy.DeleteField_management(soil_complex_table_path, fields_to_remove)
    print("Unwanted fields removed from the soil complex table.")

def main():
    extract_soil_raster()
    join_raster_attribute()
    export_raster_table()
    join_soil_complex_data()
    remove_unwanted_fields()  # Add this line to call the new function
    print("Process completed successfully. Soil raster file and soil complex data table created and cleaned.")

if __name__ == "__main__":
    arcpy.CheckOutExtension("Spatial")
    main()
    arcpy.CheckInExtension("Spatial")


Unwanted fields removed from the soil complex table.
Process completed successfully. Soil raster file and soil complex data table created and cleaned.
